In [2]:
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import pandas as pd
from IPython.display import FileLink
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import paramiko
from io import StringIO
from shapely.geometry import MultiPoint, MultiPolygon
from sklearn import preprocessing, cluster
import scipy
import scipy.cluster
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from shapely.ops import unary_union
import calendar
from datetime import datetime
import base64

/home/rajat/anaconda3/envs/gisenv/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [3]:
# Reading GeoJSON
gdf_geojson = gpd.read_file("/home/rajat/GIS Blitznow/India_Pincodes/india_pincodes.shp")

In [4]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [5]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [6]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = """with base as (
    select
        n.city_name as shipping_city
    ,   customer_pincode as shipping_pincode
    ,   date(dispatch_time) as date
    ,   count(task_id) as ofd
    from public.locus_task_brief as l
    left join application_db.node as n on l.location_id = n.locus_home_base_id
    where date_trunc('day', dispatch_time) > '2024-04-01' and dispatch_time is not null and extract(hour from dispatch_time) < 12
    group by 1, 2, 3
)
,
st1 as (
    select
        shipping_city
    ,   shipping_pincode::bigint
    ,   avg(ofd) as ofd
    from base
    group by 1, 2
)

select * from st1"""


# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)



# Now you can perform further operations with the DataFrame 'df'
print(df)

Tunnel Started
Connection Made
    shipping_city  shipping_pincode        ofd
0           Delhi            110012  14.176471
1       Bangalore            560084  20.522727
2       Bangalore            560015  13.145833
3          Jaipur            302005   3.760870
4          Jaipur            302016  26.320000
..            ...               ...        ...
473     Hyderabad            500076  29.900000
474        Mumbai            401203   1.000000
475     Hyderabad            500003  37.755556
476         Delhi            110078  41.600000
477     Hyderabad            500009  11.558824

[478 rows x 3 columns]


In [7]:
# Define the bin edges and labels for the 'ofd bucket' column
bins = [0, 10, 20, 30, 40, 50, 5000]
labels = ['0', '10', '20', '30', '40', '50']

# Use pd.cut() to categorize 'ofd' values into buckets
df['ofd bucket'] = pd.cut(df['ofd'], bins=bins, labels=labels, right=False)

In [8]:
rider_capacity = 60

# Define a function to calculate the number of riders required
def calculate_riders(ofd):
    return (ofd // rider_capacity) + (1 if ofd % rider_capacity != 0 else 0)

# Apply the function to create the new column
df['riders_required'] = df['ofd'].apply(calculate_riders)

# Make sure the 'riders_required' column is integer for discrete color scale
df['riders_required'] = df['riders_required'].astype(int)


In [9]:
# Define a function to calculate productivity
def calculate_productivity(ofd, riders_required):
    return ofd / riders_required if riders_required != 0 else 0

# Apply the function to create the new column
df['productivity'] = df.apply(lambda row: calculate_productivity(row['ofd'], row['riders_required']), axis=1)

print(df)

# Group the DataFrame by 'shipping_city' and calculate the sum of 'ofd' and 'riders_required' for each city
city_df = df.groupby('shipping_city').agg({'ofd': 'sum', 'riders_required': 'sum'}).reset_index()

# Calculate the city-wise productivity
city_df['productivity'] = city_df['ofd'] / city_df['riders_required']

print(city_df)


    shipping_city  shipping_pincode        ofd ofd bucket  riders_required  \
0           Delhi            110012  14.176471         10                1   
1       Bangalore            560084  20.522727         20                1   
2       Bangalore            560015  13.145833         10                1   
3          Jaipur            302005   3.760870          0                1   
4          Jaipur            302016  26.320000         20                1   
..            ...               ...        ...        ...              ...   
473     Hyderabad            500076  29.900000         20                1   
474        Mumbai            401203   1.000000          0                1   
475     Hyderabad            500003  37.755556         30                1   
476         Delhi            110078  41.600000         40                1   
477     Hyderabad            500009  11.558824         10                1   

     productivity  
0       14.176471  
1       20.522727  
2  

In [10]:
# Ensure the pincode column datatype is consistent
df['shipping_pincode'] = df['shipping_pincode'].astype(str)
gdf_geojson['pincode'] = gdf_geojson['pincode'].astype(str)

merged_gdf = gdf_geojson.merge(df, left_on='pincode', right_on='shipping_pincode')

In [11]:
# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Morning Slot Data", style={'text-align': 'center'}),
    dcc.Input(
        id='search_shippingcity',
        type='text',
        placeholder='Search Shipping City...',
        value='Bangalore',
        style={'width': "40%"}
    ),
    html.Div(style={'height': '20px'}),
    html.Div(id='output_container_city', children=[]),
    html.Div(id='productivity'),
    html.Div(id='riders_required'),
    html.Div(id='ofd'),
    html.Br(),
    html.Div([
        dcc.Graph(id='map1', figure={}),
        dcc.Graph(id='map2', figure={}),
        dcc.Graph(id='map3', figure={})   
    ])
])

# Callback to update city productivity
@app.callback(
    Output(component_id='productivity', component_property='children'),
    [Input(component_id='search_shippingcity', component_property='value')]
)
def update_productivity(city_slctd):
    if not city_slctd or city_slctd not in city_df['shipping_city'].unique():
        return html.Div()
    
    selected_city_data = city_df[city_df['shipping_city'] == city_slctd]
    productivity = int(selected_city_data['productivity'].sum())
    
    return html.Div(f"Productivity: {productivity}")

# Callback to update city riders
@app.callback(
    Output(component_id='riders_required', component_property='children'),
    [Input(component_id='search_shippingcity', component_property='value')]
)
def update_riders_required(city_slctd):
    if not city_slctd or city_slctd not in city_df['shipping_city'].unique():
        return html.Div()
    
    selected_city_data = city_df[city_df['shipping_city'] == city_slctd]
    riders_required = int(selected_city_data['riders_required'].sum())
    
    return html.Div(f"Riders required: {riders_required}")

# Callback to update average ofd
@app.callback(
    Output(component_id='ofd', component_property='children'),
    [Input(component_id='search_shippingcity', component_property='value')]
)
def update_riders_required(city_slctd):
    if not city_slctd or city_slctd not in city_df['shipping_city'].unique():
        return html.Div()
    
    selected_city_data = city_df[city_df['shipping_city'] == city_slctd]
    ofd = int(selected_city_data['ofd'].sum())
    
    return html.Div(f"Morning Orders: {ofd}")


# Callback to update map1
@app.callback(
    [Output(component_id='output_container_city', component_property='children'),
     Output(component_id='map1', component_property='figure')],
    [Input(component_id='search_shippingcity', component_property='value')]
)

def update_map1(city_slctd):
    if not city_slctd or city_slctd not in merged_gdf['shipping_city'].unique():
        return "", {}
    output_container_city = "Results are shown for {}:".format(city_slctd)
    
    # Filter the data for the selected city
    dff = merged_gdf[(merged_gdf["shipping_city"] == city_slctd)]
    
    # Generate centroid
    centroid = dff.geometry.centroid
    centroid_wgs84 = centroid.to_crs(epsg=4326)
    dff['geometry'] = dff['geometry'].simplify(tolerance=0.001)

    color_scale = ['#D73027', '#FC8D59', '#FEE08B', '#D9EF8B', '#91CF60', '#1A9850']  

    fig1 = px.choropleth_mapbox(
        dff,  # Use filtered data from dff
        geojson=dff.geometry,  # Pass geometry from filtered data
        locations=dff.index,   # Use index from filtered data
        title = 'Map 1: Order Buckets Pincode-Wise',
        color='ofd bucket',
        hover_data=['shipping_pincode', 'officename', 'ofd'],
        mapbox_style="carto-positron",
        center={"lat": centroid_wgs84.y.mean(), "lon": centroid_wgs84.x.mean()},
        zoom=8.5,
        opacity=0.9,
        template='plotly_dark',
        color_discrete_sequence=color_scale,  # Use discrete color scale
        category_orders={'ofd bucket': ['0', '10', '20', '30', '40', '50']}  # Ensure correct order of categories
    )

    # Update the color axis label to reflect 'ofd bucket' categories
    fig1.update_layout(coloraxis_colorbar=dict(
        title="OFD Bucket",
        tickvals=[0, 1, 2, 3, 4, 5],
        ticktext=['0', '10', '20', '30', '40', '50']
    ))
    
    return output_container_city, fig1

# Callback to update map2
@app.callback(
    Output(component_id='map2', component_property='figure'),
    [Input(component_id='search_shippingcity', component_property='value')]
)
def update_map2(city_slctd):
    if not city_slctd or city_slctd not in merged_gdf['shipping_city'].unique():
        return {}
    # Filter the data for the selected city
    dff = merged_gdf[(merged_gdf["shipping_city"] == city_slctd)]
    
    # Generate centroid
    centroid = dff.geometry.centroid
    centroid_wgs84 = centroid.to_crs(epsg=4326)
    dff['geometry'] = dff['geometry'].simplify(tolerance=0.001)
    dff['riders_required'] = dff['riders_required'].astype(str)

    fig2 = px.choropleth_mapbox(
        dff,  # Use filtered data from dff
        geojson=dff.geometry,  # Pass geometry from filtered data
        locations=dff.index,  # Use index from filtered data
        title = 'Map 2: Riders Required Pincode-Wise',
        color='riders_required',
        hover_data=['shipping_pincode', 'officename', 'ofd'],
        mapbox_style="carto-positron",
        center={"lat": centroid_wgs84.y.mean(), "lon": centroid_wgs84.x.mean()},
        zoom=8.5,
        opacity=0.9,
        template='plotly_dark'
    )
    
    return fig2

# Callback to update map3
@app.callback(
    Output(component_id='map3', component_property='figure'),
    [Input(component_id='search_shippingcity', component_property='value')]
)

def update_map3(city_slctd):
    if not city_slctd or city_slctd not in merged_gdf['shipping_city'].unique():
        return {}
    # Filter the data for the selected city
    dff = merged_gdf[(merged_gdf["shipping_city"] == city_slctd)]
    print(dff)
    # Generate centroid
    centroid = dff.geometry.centroid
    centroid_wgs84 = centroid.to_crs(epsg=4326)
    dff['geometry'] = dff['geometry'].simplify(tolerance=0.001)

    fig3 = px.choropleth_mapbox(
        dff,  # Use filtered data from dff
        geojson=dff.geometry,  # Pass geometry from filtered data
        locations=dff.index,  # Use index from filtered data
        title = 'Map 3: Productivity Pincode-Wise',
        color='productivity',
        hover_data=['shipping_pincode', 'officename', 'ofd','productivity'],
        mapbox_style="carto-positron",
        center={"lat": centroid_wgs84.y.mean(), "lon": centroid_wgs84.x.mean()},
        color_continuous_scale = 'RdYlGn',
        zoom=8.5,
        opacity=0.9,
        template='plotly_dark'
    )
    
    return fig3


if __name__ == '__main__':
    port = 8056
    dash_app_link = f"http://localhost:{port}/"
    print("Link to access Dash app externally:")
    print(dash_app_link)
    app.run(debug=True, port=port, mode='external')


Link to access Dash app externally:
http://localhost:8056/


    pincode      state   district                  officename officetype  \
9    560056  Karnataka  Bangalore     Bnagalore Viswavidalaya        S.O   
16   560011  Karnataka  Bangalore          Jayangar III Block        S.O   
17   560002  Karnataka  Bangalore  Sri Jayachamarajendra Road        S.O   
18   560017  Karnataka  Bangalore                         NAL        S.O   
19   560003  Karnataka  Bangalore                 Malleswaram        S.O   
..      ...        ...        ...                         ...        ...   
359  560001  Karnataka  Bangalore            Bangalore G.P.O.        H.O   
421  560078  Karnataka  Bangalore                   J P Nagar        S.O   
428  560016  Karnataka  Bangalore        Krishnarajapuram R S        S.O   
429  560022  Karnataka  Bangalore          Yeshwanthpur Bazar        S.O   
430  560060  Karnataka  Bangalore                     Kengeri        S.O   

                                              geometry shipping_city  \
9    POLYGON ((